In [1]:
#importar bibliotecas 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
from plotnine import *
from kmodes.kprototypes import KPrototypes

In [2]:
dados = pd.read_csv('estaticos_market.csv')
port1 = pd.read_csv('estaticos_portfolio1.csv')
port2 = pd.read_csv('estaticos_portfolio2.csv')
port3 = pd.read_csv('estaticos_portfolio3.csv')

In [3]:
dados['port1'] = dados.id.isin(port1.id)
dados['port2'] = dados.id.isin(port2.id)
dados['port3'] = dados.id.isin(port3.id)
dados['market'] = 1

In [4]:
dados = dados.loc[:, dados.isnull().mean() < 0.5]

In [5]:
# Copiar código a partir daqui 
#### Obs: A base se chama dados, mas pode ser substituído para qualquer nome, com um replace (edit > find e replace > clicar na setinha > nome antigo e novo nome)

## Informações financeiras - 20

# Grupo interessante`

dados.groupby(by='fl_mei')['port1', 'port2', 'port3', 'market'].sum()

## `Cliente MEI?`

dados.groupby(by='fl_mei').count() #['port1', 'port2', 'port3', 'market'].sum()

a = pd.get_dummies(dados.fl_mei, 'col1', 'col2')#, prefix_sep )

dados['cli_mei']= a.col1col2True

## `Cliente Simples?`

a = pd.get_dummies(dados.fl_optante_simples, 'col1', 'col2')

dados['cli_simples'] = a.col1col2True

#  `Variável de faturamento`

dados.groupby(by='de_faixa_faturamento_estimado_grupo')['port1','port2','port3', 'market'].sum()

dados.de_faixa_faturamento_estimado_grupo.unique()

# Transformando em discreta

a = {'ATE R$ 81.000,00': 1, 
     'DE R$ 81.000,01 A R$ 360.000,00' : 2,
     'DE R$ 360.000,01 A R$ 1.500.000,00' : 3, 
     'DE R$ 1.500.000,01 A R$ 4.800.000,00' : 4, 
     'DE R$ 4.800.000,01 A R$ 10.000.000,00' : 5,
     'DE R$ 10.000.000,01 A R$ 30.000.000,00' : 6,
     'DE R$ 30.000.000,01 A R$ 100.000.000,00' : 7,
     'DE R$ 100.000.000,01 A R$ 300.000.000,00': 8, 
     'DE R$ 300.000.000,01 A R$ 500.000.000,00' : 9, 
     'DE R$ 500.000.000,01 A 1 BILHAO DE REAIS' : 10, 
     'ACIMA DE 1 BILHAO DE REAIS' : 11} 

dados['faturamento_cat'] = dados['de_faixa_faturamento_estimado_grupo'].map(a)

# Tratano nulos 

# Verifica se é nulo e mei
#dados['de_faixa_faturamento_estimado_grupo'].isnull() & dados['fl_mei'] == True

## Substituição de faturamento para clientes MEI

dados.faturamento_cat = np.where(dados['de_faixa_faturamento_estimado_grupo'].isnull() & dados['fl_mei'] == True, 1, dados.faturamento_cat)

dados.faturamento_cat = np.where(dados['faturamento_cat'].isnull() & dados['fl_optante_simei'] == True, 1, dados.faturamento_cat)

## Substituição de faturamento para clientes simples

dados.faturamento_cat = np.where(dados['faturamento_cat'].isnull() & dados['fl_optante_simples'] == True, 2, dados.faturamento_cat)

dados.faturamento_cat = np.where(dados['faturamento_cat'].isnull() & dados['fl_me'] == True, 2, dados.faturamento_cat)

## Verificando nulos restantes e suas características 

dados.faturamento_cat.isnull().sum()

dados.setor[dados['faturamento_cat'].isnull()].groupby(by=dados.setor).count()

#dados[(dados['setor'] == 'SERVIÇO') & (dados['idade_emp_cat'] == '<= 1')].groupby(by='faturamento_cat').count()

dados.setor[dados['faturamento_cat'].isnull()].groupby(by=dados.idade_emp_cat).count()

#dados[dados['idade_emp_cat'] == '<= 1'].groupby(by='faturamento_cat').count()

dados.faturamento_cat = np.where(dados['faturamento_cat'].isnull(), 1.5, dados.faturamento_cat)

# Input final 
#### Coloca as empresas faltantes no grupo 1.5, porque os clientes possuem em geral menos de 1 ano e são dos setores de Comercio ou Serviços. Ambas as características estão majoritariamente nas faixas de faturamento 1 e 2.

dados.faturamento_cat = np.where(dados['faturamento_cat'].isnull(), 1.5, dados.faturamento_cat)

## `Quantidade de filiais` 

#df.groupby(by='qt_filiais').count() #agg({'id' : np.size})

bins = [0, 1, 2, 5, 9, 65,  np.inf]
names = ['0-1','1-2', '2-5', '5-9', '9-65', '65+']
dados['qt_filiais_range'] = pd.cut(dados['qt_filiais'], bins, labels=names, include_lowest=True)

dados.groupby(by='qt_filiais_range')['port1', 'port2', 'port3', 'market'].sum()

## `Probabilidade de estar operando`

dados.groupby(by='de_nivel_atividade')['port1', 'port2', 'port3', 'market'].sum().head(17)

## `Verdadeiro caso realize atividades tributáveis`

dados.groupby(by='fl_passivel_iss')['port1', 'port2', 'port3', 'market'].sum()

dados[dados.fl_passivel_iss.isnull()].isnull().sum()

dados[dados.fl_passivel_iss.isnull()].groupby(by='idade_emp_cat')['port1', 'port2', 'port3', 'market'].sum()

dados[dados.idade_emp_cat == '> 20'].groupby(by='fl_passivel_iss')['port1', 'port2', 'port3', 'market'].sum()

dados[dados.fl_passivel_iss.isnull()].groupby(by='vl_faturamento_estimado_aux')['port1', 'port2', 'port3', 'market'].sum()

dados[dados.vl_faturamento_estimado_aux == 210000.0].groupby(by='fl_passivel_iss')['port1', 'port2', 'port3', 'market'].sum()

dados[dados.fl_passivel_iss.isnull()].groupby(by='de_faixa_faturamento_estimado')['port1', 'port2', 'port3', 'market'].sum()

dados[dados.de_faixa_faturamento_estimado == 'DE R$ 81.000,01 A R$ 360.000,00'].groupby(by='fl_passivel_iss')['port1', 'port2', 'port3', 'market'].sum()

## Variáveis seleciondas 

features = dados[['id', 'port1', 'port2', 'port3', 'market','faturamento_cat','cli_mei', 'cli_simples', 'qt_filiais', 'setor']]

In [6]:
dados.setor = np.where(dados['setor'].isnull(), 'OUTROS', dados.setor)
dados.nm_segmento = np.where(dados['nm_segmento'].isnull(), 'OUTROS', dados.nm_segmento)

### Criando a função

#### Modelagem
> **1.** Escolha as variáveis a serem usadas  
> **2.** Escolha o portfolio  
> **3.** Calcule o centroide do portfolio  
> **4.** Calcule a distancia de cada empresa do market ao portfolio

#### Avaliação
> **1.** Computar o within.ss  
> **2.** Fixar um numero de leads a serem retornados e verificar quantos clientes seriam retornados


In [7]:
def cluster_foda(df, var_cat, var_num, port, n_leads = 100):
    
    # Verifica se tem nulos nas variaveis selecionadas.
    nulos = df[var_num + var_cat].isna().sum()
    if any(nulos > 0):
        print("tira os nulos caraio")
        return nulos[nulos>0]
    
    treino = df[df[port] == 1][var_cat + var_num]
    teste = df[var_cat + var_num].copy()
    
    # Calcula o centroide dos clientes
    kproto = KPrototypes(n_clusters=1, init='Cao', verbose=2)
    index_cat = list(range(len(var_cat)))
    clusters, distances = kproto.fit_predict(treino, categorical=index_cat)
    
    # Faz prediçao das distancias pro market.
    clusters_teste, distances_teste = kproto.predict(teste, categorical=index_cat)
    
    # Cria variavel de distancia na base do market
    teste['dist'] = distances_teste
    teste['portfolio'] = df[port]
    leads = teste.sort_values('dist').head(n_leads)
    
    result = dict()
    result['leads'] = leads
    result['centroide'] = kproto.cluster_centroids_
    result['within.dist'] = kproto.cost_
    
    return result 

In [12]:
x = cluster_foda(df = dados,
                 var_cat = ['setor', 'nm_segmento', 'faturamento_cat','cli_mei', 'cli_simples'],
                 var_num = ['faturamento_cat', 'qt_filiais'],
                 port = 'port3',
                 n_leads = 10000)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 1, iteration: 1/100, moves: 0, ncost: 39452.432970290145
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 2, iteration: 1/100, moves: 0, ncost: 39452.432970290145
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 3, iteration: 1/100, moves: 0, ncost: 39452.432970290145
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 4, iteration: 1/100, moves: 0, ncost: 39452.432970290145
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 5, iteration: 1/100, moves: 0, ncost: 39452.432970290145
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 6, iteration: 1/100, moves: 0, ncost: 39452.432970290145
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 7, iteration: 1/100, moves: 0, ncost: 39452.432970290145
Init: 

In [13]:
x['leads']

,setor,nm_segmento,faturamento_cat,cli_mei,cli_simples,faturamento_cat,qt_filiais,dist,portfolio
35369,SERVIÇO,ADMINISTRACAO PUBLICA DEFESA E SEGURIDADE SOCIAL,7.0,0,0,7.0,3,0.223396,False
319330,SERVIÇO,ADMINISTRACAO PUBLICA DEFESA E SEGURIDADE SOCIAL,7.0,0,0,7.0,3,0.223396,False
79017,SERVIÇO,ADMINISTRACAO PUBLICA DEFESA E SEGURIDADE SOCIAL,7.0,0,0,7.0,3,0.223396,False
264501,SERVIÇO,ADMINISTRACAO PUBLICA DEFESA E SEGURIDADE SOCIAL,7.0,0,0,7.0,3,0.223396,False
329725,SERVIÇO,ADMINISTRACAO PUBLICA DEFESA E SEGURIDADE SOCIAL,7.0,0,0,7.0,3,0.223396,False
...,...,...,...,...,...,...,...,...,...
60153,COMERCIO,COMERCIO; REPARACAO DE VEICULOS AUTOMOTORES E ...,4.0,0,0,4.0,1,25.607656,False
64676,COMERCIO,COMERCIO; REPARACAO DE VEICULOS AUTOMOTORES E ...,4.0,0,0,4.0,1,25.607656,False
385879,SERVIÇO,ALOJAMENTO E ALIMENTACAO,4.0,0,1,4.0,1,25.607656,False
385871,COMERCIO,COMERCIO; REPARACAO DE VEICULOS AUTOMOTORES E ...,4.0,0,0,4.0,1,25.607656,False


In [14]:
x['leads'].portfolio.sum()

246

In [49]:
dados[['setor', 'nm_segmento', 'cli_mei', 'faturamento_cat']]

,setor,nm_segmento,cli_mei,faturamento_cat
0,CONSTRUÇÃO CIVIL,CONSTRUCAO,0,4.0
1,SERVIÇO,ALOJAMENTO E ALIMENTACAO,1,2.0
2,SERVIÇO,TRANSPORTE ARMAZENAGEM E CORREIO,1,1.0
3,SERVIÇO,OUTRAS ATIVIDADES DE SERVICOS,0,2.0
4,SERVIÇO,ATIVIDADES PROFISSIONAIS CIENTIFICAS E TECNICAS,0,2.0
...,...,...,...,...
462293,COMERCIO,COMERCIO; REPARACAO DE VEICULOS AUTOMOTORES E ...,1,1.0
462294,SERVIÇO,ATIVIDADES ADMINISTRATIVAS E SERVICOS COMPLEME...,0,2.0
462295,SERVIÇO,SAUDE HUMANA E SERVICOS SOCIAIS,0,2.0
462296,COMERCIO,COMERCIO; REPARACAO DE VEICULOS AUTOMOTORES E ...,0,3.0


In [50]:
names = ['setor', 'nm_segmento', 'cli_mei', 'faturamento_cat']

In [52]:
data =  {'Variavel':names, 'centroide':x['centroide']} 

In [57]:
names
len(x['centroide'])

2

In [58]:
len(names)

4

In [77]:
pd.Series(x['centroide'][0][0])

0    0.000000
1    7.158491
dtype: float64

In [78]:
pd.Series(x['centroide'][1][0])

0                                             SERVIÇO
1    ADMINISTRACAO PUBLICA DEFESA E SEGURIDADE SOCIAL
dtype: object

In [54]:
pd.DataFrame(data)

ValueError: arrays must all be same length